# Generator

In [ ]:
from matplotlib import pyplot as plt
import os, sys
from glob import glob
from matplotlib import pyplot

from PIL import Image
from PIL import EpsImagePlugin
import math

from PIL import ImageFilter
import numpy as np

from lxml import etree
import xml.etree.ElementTree as xml
import random
import cv2


In [ ]:
def preSign(size):
    sign_data = glob('./data/gen/sign/*.png')
    size = size, size

    for s in sign_data:
        clssName = s.split('sign/')[1]

        if 'DT_177' in s:
            img = cv2.imread(s)
            kernel = np.ones((3, 3), np.uint8)
            img = cv2.erode(img, kernel, iterations=1)

            cv2.imwrite('./data/gen/new/'+clssName, img)

        else:
            img = cv2.imread(s)
            kernel = np.ones((7, 7), np.uint8)
            img = cv2.erode(img, kernel, iterations=1)

            cv2.imwrite('./data/gen/new/'+clssName, img)


    
    new_data = glob('./data/gen/new/*.png')
    print(size)
    for m in new_data:
        im = Image.open(m)
        im.thumbnail(size, Image.ANTIALIAS)
        im = im.convert("RGBA")
        datas = im.getdata()
        newData = []

        for item in datas:
            if item[0] > 200 and item[1] > 200 and item[2] > 200:
                newData.append((item[0], item[1], item[2], 0))
            else:
                newData.append(item)

            im.putdata(newData)
            im.save(m)

            

def ChangeColor(inputImg, r2, g2, b2):

    im = Image.open(inputImg)

    im = np.array(im)
    ret, im = cv2.threshold(im, 247, 255, cv2.THRESH_TOZERO)
    data = im
    
    print(inputImg)

    imName = inputImg.split('gen/')[1]
    imName = imName.split(".png")[0]

    r1, g1, b1 = 0, 0, 0  # Original value
    r2, g2, b2 = r2, g2, b2  # Value that we want to replace it with

    red, green, blue = data[:, :, 0], data[:, :, 1], data[:, :, 2]
    mask = (red == r1) & (green == g1) & (blue == b1)
    data[:, :, :3][mask] = [r2, g2, b2]

    im = Image.fromarray(data)
    imDir = "./data/gen/" + imName + '.png'
    im.save(imDir)

    return imDir



def preDraw():
    draw_data = glob('./Desktop/ETRI/DrawGen/data/standard/draw/*.png')

    for  i in draw_data:
        im = cv2.imread(i)
        ret, imGray = cv2.threshold(im,150,255, cv2.THRESH_BINARY_INV)

        gray = cv2.cvtColor(imGray, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(i, gray)


def genData(dr_fileName):
    # dr_fileName = 'C:/Users/USER/PycharmProjects/DrawGen/data/standard/draw/10f.png'

    l_img = cv2.imread(dr_fileName)
    h, w, d = l_img.shape

    sign_data = glob('./Desktop/ETRI/DrawGen/data/standard/sign/*.png')

    signList = []
    # set a color RGB
    r2, g2, b2 = 255,255,255

    # set an amoumnt of sign #
    amountList = []
    for a in range(10):
        amountList.append(random.randrange(0, 5))


    total = sum(amountList)

    new_OriFileName = 'GEN_' + str(int(random.random() * 100000))
    new_fileName = new_OriFileName + '.png'
    new_dirFileName = './Desktop/ETRI/DrawGen/data/gen/' + new_fileName

    root = etree.Element("annotation")

    folder = etree.SubElement(root, "folder").text = 'dataset'
    filename = etree.SubElement(root, "filename").text = new_fileName
    path = etree.SubElement(root, "path").text = new_dirFileName

    source = etree.SubElement(root, "source")
    database = etree.SubElement(source, "database").text = 'Unknown'

    size = etree.SubElement(root, "size")
    width = etree.SubElement(size, "width").text = str(w)
    height = etree.SubElement(size, "height").text = str(h)
    depth = etree.SubElement(size, "depth").text = str(d)

    segmented = etree.SubElement(root, "segmented").text = '0'

    print(total) 
    print(new_dirFileName)  

    xmin = random.sample(range(50, w), total)
    ymin = random.sample(range(50, h), total)

    location = list(zip(xmin, ymin))

    for i, j in zip(sign_data, amountList):

        signDir = i
        clssName = signDir.split('/sign')[1]
        clssName = clssName .split(clssName[0])[1]
        clssName = clssName.split('.png')[0]

        signList.append(clssName)

        imDir = ChangeColor(signDir, r2, g2, b2)

        s_img = cv2.imread(imDir, -1)
        alpha_s = s_img[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s

        for k in range(0, j):

            x1, y1 = location.pop()
            x2 = x1 + s_img.shape[1]
            y2 = y1 + s_img.shape[0]

            print(x1, x2, y1, y2)


            for c in range(0, 3):
                l_img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] + alpha_l * l_img[y1:y2, x1:x2, c])

            obj = etree.SubElement(root, "object")
            name = etree.SubElement(obj, "name").text = clssName
            poset = etree.SubElement(obj, "pose").text = 'Unspecified'
            truncated = etree.SubElement(obj, "truncated").text = '0'
            difficult = etree.SubElement(obj, "difficult").text = '0'

            bndbox = etree.SubElement(obj, "bndbox")
            xmin = etree.SubElement(bndbox, "xmin").text = str(x1)
            ymin = etree.SubElement(bndbox, "ymin").text = str(y1)
            xmax = etree.SubElement(bndbox, "xmax").text = str(x2)
            ymax = etree.SubElement(bndbox, "ymax").text = str(y2)


    cv2.imwrite(new_dirFileName, l_img)
    tree = etree.ElementTree(root)

    with open('./Desktop/ETRI/DrawGen/data/standard/draw/' + new_OriFileName + '.xml', "bw") as fh:
        tree.write(fh)

In [ ]:
#preSign()
#preDraw()


#drawData = glob('./Desktop/ETRI/DrawGen/data/standard/draw/*')


#for index, val in enumerate(drawData):
#    print(index)
#    try:
#        genData(val)
#    except:
#        continue

# Reverse from csv to draw image 

In [112]:
import csv

import os
from glob import glob
import os.path
import cv2

In [140]:
# set a rgb value
r2,g2,b2 = 1,1,1


i = './data/testset/out/1-f_pan.png.csv'
f = open(i, 'r', encoding='utf-8')
rdr = csv.reader(f)
    
# get draw image names
Origindraw = i.split('.csv')[0]
Newdraw = './data/testset/new_draw/b2.png'

new_dir = Newdraw.split('new_draw/')[1]
new_dir = new_dir.split('.png')[0]
new_dir = './data/testset/reverse/out/'+new_dir+'.png'

l_img = cv2.imread(Newdraw) # read new daraw image

l_img_height, l_img_width, ch = l_img.shape
l_img_Scale = l_img_width/l_img_height 

origin_img = cv2.imread(Origindraw) # read origin daraw image
origin_img_height, origin_img_width, ch = origin_img.shape
origin_img_Scale = origin_img_width/origin_img_height     

scal_heigh = round(l_img_height/origin_img_height)
scal_width = round(l_img_width/origin_img_width)


for k,line in enumerate(rdr):
    
    clssName = line[0]
    xmin = int(line[2])
    ymin = int(line[3])
    xmax = int(line[4])
    ymax = int(line[5])
     
    
    if k == 0 :
        size = (xmax-xmin) * scal_width
        
        preSign(size)

        pass
    
    signDir= ('./data/gen/new/'+clssName+'.png')
    
    imName = ChangeColor(signDir,r2,g2,b2)
    s_img =  cv2.imread(imName,-1)
    
    new_x1 = xmin * scal_width
    new_y1 = ymin * scal_heigh
    new_x2 = xmin + s_img.shape[1]
    new_y2 = ymin + s_img.shape[0]
    

    alpha_s = s_img[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s
     

    for c in range(0, 3):
        l_img[new_y1:new_y2, new_x1:new_x2, c] = (alpha_s * s_img[:, :, c] + alpha_l * l_img[new_y1:new_y2, new_x1:new_x2, c])


cv2.imwrite(new_dir, l_img)        
f.close()   
    
    

1069 1623
1034 1613
1 1
804 820 455 465
(16, 16)
DT_131
./data/gen/new/DT_131.png
804 820 455 471
891 901 458 467
DT_131
./data/gen/new/DT_131.png
891 907 458 474
876 888 451 461
DT_131
./data/gen/new/DT_131.png
876 892 451 467
886 898 463 472
DT_131
./data/gen/new/DT_131.png
886 902 463 479
873 885 455 465
DT_131
./data/gen/new/DT_131.png
873 889 455 471
960 971 465 478
DT_131
./data/gen/new/DT_131.png
960 976 465 481
952 964 459 471
DT_131
./data/gen/new/DT_131.png
952 968 459 475
1164 1184 513 531
DT_143
./data/gen/new/DT_143.png
1164 1180 513 529
1167 1183 522 538
DT_143
./data/gen/new/DT_143.png
1167 1183 522 538
1089 1109 25 38
DT_150
./data/gen/new/DT_150.png
1089 1105 25 34
1085 1108 30 42
DT_150
./data/gen/new/DT_150.png
1085 1101 30 39
1538 1559 258 270
DT_150
./data/gen/new/DT_150.png
1538 1554 258 267
434 454 504 516
DT_150
./data/gen/new/DT_150.png
434 450 504 513
124 142 928 939
DT_150
./data/gen/new/DT_150.png
124 140 928 937
454 477 849 862
DT_177
./data/gen/new/DT_177.

# Make annntation file for MAP
## ground-truth

In [146]:
from xml.etree.ElementTree import parse


xml_data = glob('./data/testset/ground-truth/input/*.xml')
for i in xml_data:

    xminList = []
    yminList = []
    xmaxList = []
    ymaxList = []

    fileName = i.split('input/')[1]
    fileName = fileName.split('.png')[0]
    
    tree = parse(i)
    root = tree.getroot()
    kids = root.getchildren()

    obj = root.findall("object")
    clss = [x.findtext("name") for x in obj]

    fileName = root[1].text

    for neighbor in root.iter('bndbox'): 
        xminList.append(neighbor.findtext("xmin"))
        yminList.append(neighbor.findtext("ymin"))
        xmaxList.append(neighbor.findtext("xmax"))
        ymaxList.append(neighbor.findtext("ymax"))


    f = open('./data/testset/ground-truth/'+fileName+'.txt', 'w')
    for i in range (len(xminList)): 
        f.write(clss[i]+' '+xminList[i]+' '+yminList[i]+' '+xmaxList[i]+' '+ymaxList[i]+'\n')
    f.close()

## detection-results

In [148]:
xml_data = glob('./data/testset/out/*.csv')
for i in xml_data:

    fileName = i.split('out/')[1]
    fileName = fileName.split('.png')[0]
    
    f = open(i, 'r', encoding='utf-8')
    rdr = csv.reader(f)
    
    f = open('./data/testset/detection-results/'+fileName+'.txt', 'w')
    for line in rdr:

        f.write(line[0]+' '+ line[1] +' '+line[2]+' '+line[3]+' '+line[4]+' '+line[5]+'\n')
    f.close()
